## Noise Estimation Brute Force Approach

- For a single noise value, have multiple simulations for that and that will show the distribution of possible values at that noise level
-  Note that each noise value samples from a distribution already, and the same noise value is going to have different results each time due to this
- References: MonteCarlo from monte_carlo.py and example in 2024PHMTutorial

In [ ]:
from progpy.models import ThrownObject
import numpy as np
import matplotlib.pyplot as plt
from progpy.predictors import MonteCarlo
from scipy.optimize import minimize

# Pick a process and measurement noise as a goal --> then use that as "real" data
PROCESS_NOISE = 1.8
MEASUREMENT_NOISE = 0.4

m = ThrownObject(process_noise=PROCESS_NOISE, measurement_noise=MEASUREMENT_NOISE)
initial_state = m.initialize()

simulated_results = m.simulate_to(8, save_freq=1)

times = simulated_results.times
inputs = simulated_results.inputs
outputs = simulated_results.outputs

def future_loading(t, x=None):
    return {}

STEP_SIZE = 1
NUM_SAMPLES = 100
PREDICTION_HORIZON = times[-1]
SAVE_FREQ = 1

og_mc = MonteCarlo(m)
og_mc_results = og_mc.predict(initial_state, future_loading, dt=STEP_SIZE, horizon=PREDICTION_HORIZON, save_freq=SAVE_FREQ, n_samples=NUM_SAMPLES)

In [ ]:
PROCESS_NOISE_GUESS = 3
MEASUREMENT_NOISE_GUESS = 2

m = ThrownObject(process_noise=PROCESS_NOISE_GUESS, measurement_noise=MEASUREMENT_NOISE_GUESS)
initial_state = m.initialize()

mc = MonteCarlo(m)
mc_results = mc.predict(initial_state, future_loading, dt=STEP_SIZE, horizon=PREDICTION_HORIZON, save_freq=SAVE_FREQ, n_samples=NUM_SAMPLES)
mc_results.outputs

In [ ]:
# Plot MC guess results
for i in range(NUM_SAMPLES):
    data_pos = [state['x'] for state in mc_results.outputs[i]]
    plt.plot(mc_results.times[:len(data_pos)], data_pos, color="orange", alpha=0.2)

# Plot real data results
for i in range(NUM_SAMPLES):
    data_pos = [state['x'] for state in og_mc_results.outputs[i]]
    plt.plot(og_mc_results.times[:len(data_pos)], data_pos, color="teal", alpha=0.2)

# Plot model with no noise
m = ThrownObject(process_noise=0, measurement_noise=0)
simulated_results = m.simulate_to(times[-1], save_freq=1)
no_noise_pos = [state['x'] for state in simulated_results.outputs]
plt.plot(times, no_noise_pos, color="black", linestyle="--", label="No noise simulation")

# Plot real data mean
og_mean_pos = [state['x'] for state in og_mc_results.outputs.mean]
plt.plot(times, og_mean_pos, color="blue", linestyle="--", label="Real data mean")

# Plot MC guess mean
mc_mean_pos = [state['x'] for state in mc_results.outputs.mean]
plt.plot(times, mc_mean_pos, color="tab:orange", linestyle="--", label="MC guess mean")

plt.legend()
plt.title("Monte Carlo simulation for one noise value of Thrown Object")
plt.xlabel("Time (s)")
plt.ylabel("Position (m)")
plt.show()

error = np.mean((np.array(og_mean_pos) - np.array(mc_mean_pos))**2)
print("MSE error between real data and MC guess mean: ", error)

## Distrbutions from Brute Force Approach
- At one time step (like the last one of 8 or let's say 6), get the distribution and compare to the data (which is also a distribution and compare that - get the error value between them and use that to optimize)
- With other noise values (like 5 or 10% below or above optimized number), see how sensitivity changes (mean and variance) after the optimization
   - Helps user understand how precise they need to be with noise number

In [ ]:
for x in range(9):
    dist = []
    for i in mc_results.outputs:
        if len(i) > x:
            dist.append(i[x]['x'])

    og_dist = []
    for i in og_mc_results.outputs:
        if len(i) > x:
            og_dist.append(i[x]['x'])

    print("MC guess distribution mean: ", np.mean(dist))
    print("MC guess distribution std: ", np.std(dist))

    print("\nReal data distribution mean: ", np.mean(og_dist))
    print("Real data distribution std: ", np.std(og_dist))

    print("\nReal data - MC guess difference mean: ", np.mean(og_dist) - np.mean(dist))
    print("Real data - MC guess difference std: ", np.std(og_dist) - np.std(dist))

    fig, axes = plt.subplots(1, 2, figsize=(12, 4))

    axes[0].hist(og_dist, color="teal")
    axes[1].hist(dist, color="tab:orange")

    axes[0].set_title("Real data")
    axes[0].set_xlabel("Position (m)")
    axes[0].set_ylabel("Count")

    axes[1].set_title("MC guess")
    axes[1].set_xlabel("Position (m)")
    axes[1].set_ylabel("Count")

    fig.suptitle("Distribution at time " + str(x))
    plt.tight_layout()
    plt.show()

In [ ]:
# Real data
[state['x'] for state in og_mc_results.outputs.mean]

In [ ]:
# MC guess
[state['x'] for state in mc_results.outputs.mean]

In [ ]:
# Minimize the error between the simulated data and the original data

import numpy as np
from scipy.optimize import minimize

import warnings
warnings.filterwarnings("ignore")

original_data = [state['x'] for state in og_mc_results.outputs.mean]

def monte_carlo_simulation(process_noise, measurement_noise):
    m = ThrownObject(process_noise=process_noise, measurement_noise=measurement_noise)
    initial_state = m.initialize()

    mc = MonteCarlo(m)
    mc_results = mc.predict(initial_state, future_loading, dt=STEP_SIZE, horizon=PREDICTION_HORIZON, save_freq=SAVE_FREQ, n_samples=NUM_SAMPLES)
    
    simulated_data = [state['x'] for state in mc_results.outputs.mean]
    
    return simulated_data

def optimization_fcn(params):
    process_noise = params[0]
    measurement_noise = params[1]
    
    simulated_data = monte_carlo_simulation(process_noise, measurement_noise)
    
    # Calculate the MSQ error between simulated and original data (mean and std deviation)
    mean_error = (np.mean(simulated_data) - np.mean(original_data))**2
    std_error = (np.std(simulated_data) - np.std(original_data))**2
    
    total_error = mean_error + std_error

    print("Process Noise: ", process_noise)
    print("Measurement Noise: ", measurement_noise)
    print("Total Error: ", total_error, "\n")
    
    return total_error

In [ ]:
# Actual is [1.8, 0.4]
initial_guess = [3, 2]

# Run the optimization using scipy minimize
result = minimize(optimization_fcn, initial_guess, method='Nelder-Mead', bounds=((0, 5), (0, 5)))

# Display the optimized process and measurement noise
print("\nOptimized process noise:", result.x[0])
print("Optimized measurement noise:", result.x[1])

In [ ]:
# Actual is [1.8, 0.4]
initial_guess = [3, 2]

# Run the optimization using scipy minimize
result = minimize(optimization_fcn, initial_guess, method='Powell', bounds=((0, 5), (0, 5)))

# Display the optimized process and measurement noise
print("\nOptimized process noise:", result.x[0])
print("Optimized measurement noise:", result.x[1])

In [ ]:
# Could possibly specify the frequency at which we compare (like a eval freq) instead of doing at every time step (could also investigate
# coarse to fine optimization section)

# Look at other measures of distributional similarity - see link in chat

## Coarse to fine optimization
- Running a few samples (either running 15 mc simulations or running to time 2 seconds)
- Measure runtime, computational power, accuracy of results (since we know our actual noise values since we made the data)

In [ ]:
# Explore other optimization methods